<a href="https://colab.research.google.com/github/Matam-Rohith/NLP/blob/main/NLP_LAB_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
#[A] Data Preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Small dataset of English-to-French translations
data = [
    ("hello", "bonjour"), ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir")
]

# Separate English and French sentences
english_sentences, french_sentences = zip(*data)

# Tokenize and preprocess data
def tokenize_and_pad(sentences, tokenizer=None, max_len=10):
    if not tokenizer:
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')
    return padded_sequences, tokenizer

# Tokenize English and French data
english_sequences, eng_tokenizer = tokenize_and_pad(english_sentences)
french_sequences, fr_tokenizer = tokenize_and_pad(french_sentences)

# Vocabulary sizes
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fr_vocab_size = len(fr_tokenizer.word_index) + 1


In [21]:
#(B) Build Seq2Seq Model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# Model parameters
embedding_dim = 64
lstm_units = 64
max_seq_len = 10

# Encoder
encoder_inputs = Input(shape=(max_seq_len,))
encoder_embedding = Embedding(eng_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_seq_len,))
decoder_embedding = Embedding(fr_vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(fr_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [22]:
#(C) Preparing the Data for Training
# Prepare shifted target sequences for training
french_sequences_target = np.zeros_like(french_sequences)
french_sequences_target[:, :-1] = french_sequences[:, 1:]  # Shifted target sequence


In [23]:
#(D) Train the Model on the Dataset
# Train the Seq2Seq model
model.fit(
    [english_sequences, french_sequences], french_sequences_target[:, :, np.newaxis],
    batch_size=16,
    epochs=100,
    validation_split=0.2
)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - loss: 2.6341 - val_loss: 2.5928
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 2.5974 - val_loss: 2.5493
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - loss: 2.5593 - val_loss: 2.5024
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - loss: 2.5183 - val_loss: 2.4500
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 2.4727 - val_loss: 2.3903
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 2.4208 - val_loss: 2.3211
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 2.3608 - val_loss: 2.2396
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step - loss: 2.2904 - val_loss: 2.1429
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 2.2070 - val_loss: 2.0272
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 2.1076 - val_loss: 1.8888
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 1.9887 - val_loss: 1.7242
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 1.8473 - val_lo

In [24]:
#(E) Inference Setup for Translation
# Encoder model for inference
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder model for inference
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [25]:
# Translation function
def translate(sentence):
    # Encode the input sentence
    input_seq = pad_sequences(eng_tokenizer.texts_to_sequences([sentence]), maxlen=max_seq_len, padding='post')
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with the start token
    target_seq = np.zeros((1, 1))

    # Generate the translation sentence
    translated_sentence = ""
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the highest probability token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = fr_tokenizer.index_word.get(sampled_token_index, '?')
        translated_sentence += ' ' + sampled_word

        # Stop condition: max length or end token ('eos')
        if sampled_word == 'eos' or len(translated_sentence.split()) > max_seq_len:
            stop_condition = True

        # Update the target sequence with the sampled word
        target_seq = np.array([[sampled_token_index]])
        states_value = [h, c]

    return translated_sentence.strip()

# Test translation
print(translate("hello"))
print(translate("thank you"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
? ? ? ? ? ? ? ? ? ? ?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
? ? ? ? ? ? ? ? ? ? ?
